In [ ]:
import io
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import urllib

from IPython.display import display
from skimage import io as io_url

In [ ]:
# Image resolution
N = 99

# Create image widgets
image_spatial  = widgets.Image(format='png', width=300, height=300, description='Spatial')
image_freq     = widgets.Image(format='png', width=300, height=300, description='Frequency')
sidebyside     = widgets.HBox([image_spatial, image_freq])

# Create slider widgets
slider_rho     = widgets.IntSlider  (value=10, min=0, max=int((N-1)/2),        description='Rho (pixels)')
slider_theta   = widgets.FloatSlider(value=0,  min=0, max=2*np.pi, step=0.001, description='Theta (rads)')

buf            = io.BytesIO()

def example1(change):
    rho   = slider_rho.value
    theta = slider_theta.value
        
    x = np.round(rho*np.cos(theta))
    y = np.round(rho*np.sin(theta))
        
    fimg = np.zeros((N, N))
    fimg[int(N/2+x), int(N/2+y)] = 0.5
    fimg[int(N/2-x), int(N/2-y)] = 0.5
    fimg[int(N/2), int(N/2)] = fimg[int(N/2), int(N/2)] + 1
        
    img = np.fft.ifft2(np.fft.ifftshift(fimg))
    
    buf.seek(0)
    tmp = PIL.Image.fromarray(255*np.abs(img)*(N**2)/2)
    tmp = tmp.convert('L')
    tmp.save(buf, 'png')
    image_spatial.value = buf.getvalue()
        
    buf.seek(0)
    tmp = PIL.Image.fromarray(255*np.abs(fimg))
    tmp = tmp.convert('L')
    tmp.save(buf, 'png')
    image_freq.value = buf.getvalue()

                          
slider_rho.observe  (example1, names='value')
slider_theta.observe(example1, names='value')

example1(0)

display(sidebyside)
display(slider_rho)
display(slider_theta)

In [ ]:
# Image resolution
N = 99

# Create image widgets
image2_spatial = widgets.Image(format='png', width=300, height=300, description='Spatial')
image2_freq    = widgets.Image(format='png', width=300, height=300, description='Frequency')
sidebyside     = widgets.HBox([image2_spatial, image2_freq])

# Create slider/select widgets
slider_size    = widgets.IntSlider(value=25, min=0, max=int((N-1)/2), description='Size')
select_filter  = widgets.Select(options=['Box', 'Gaussian'], value='Box')

buf            = io.BytesIO()

def example2(change):
    x = slider_size.value
    
    if (select_filter.value=='Gaussian'):
        x = 1/(x+1)
        xv, yv = np.meshgrid(np.linspace(-1,1,N), np.linspace(-1,1,N))
        img = np.exp(-(xv**2 + yv**2)/(x**2))/x
    elif (select_filter.value=='Box'):
        img = np.zeros((N,N))
        img[int(N/2-x):int(N/2+x+1),int(N/2-x):int(N/2+x+1)] = 1
    
    fimg = np.fft.fftshift(np.fft.fft2(img))
    fimg = np.abs(fimg)
        
    buf.seek(0)
    tmp = PIL.Image.fromarray(255*img/img.max())
    tmp = tmp.convert('L')
    tmp.save(buf, 'png')
    image2_spatial.value = buf.getvalue()

    buf.seek(0)
    tmp = PIL.Image.fromarray(255*fimg/fimg.max())
    tmp = tmp.convert('L')
    tmp.save(buf, 'png')
    image2_freq.value = buf.getvalue()
        

slider_size.observe(example2, names='value')
select_filter.observe(example2, names='value')

example2(0)

display(sidebyside)
display(slider_size)
display(select_filter)

In [ ]:
# Create image widgets
image3_spatial = widgets.Image(format='png', width=300, height=300, description='Spatial')
image3_freq    = widgets.Image(format='png', width=300, height=300, description='Frequency')
sidebyside     = widgets.HBox([image3_spatial, image3_freq])

# Create slider/select widgets
slider_inner   = widgets.FloatSlider(value=0,      min=0, max=1,      step=0.01, description='Inner radius')
slider_outer   = widgets.FloatSlider(value=1.44/2, min=0, max=1.44/2, step=0.01, description='Outer radius')

buf            = io.BytesIO()

orig_img = io_url.imread('http://16385.courses.cs.cmu.edu/fall2020/assets/images/data/scotty.jpg')
orig_img = orig_img[:,:,1]

x = np.fft.fftfreq(orig_img.shape[0]);
y = np.fft.fftfreq(orig_img.shape[1]);

xv, yv = np.meshgrid(x, y)
xv = np.fft.fftshift(xv)
yv = np.fft.fftshift(yv)

def on_value_change3(change):
    mask = (np.sqrt(xv**2 + yv**2) < slider_outer.value) & \
           (np.sqrt(xv**2 + yv**2) >= slider_inner.value)
    mask = np.float32(mask)
        
    fimg = np.fft.fftshift(np.fft.fft2(orig_img))
    fimg = fimg*mask
        
    img = np.fft.ifft2(np.fft.ifftshift(fimg))
    img = np.abs(img)
    
    fimg = np.abs(fimg)

    buf.seek(0)
    tmp = PIL.Image.fromarray(255*img/(img.max()+0.01))
    tmp = tmp.convert('L')
    tmp.save(buf, 'png')
    image3_spatial.value = buf.getvalue()
    
    buf.seek(0)
    tmp = PIL.Image.fromarray(255*np.log(0.0001*fimg + 1))
    tmp = tmp.convert('L')
    tmp.save(buf, 'png')
    image3_freq.value = buf.getvalue()

    
slider_inner.observe(on_value_change3, names='value')
slider_outer.observe(on_value_change3, names='value')

on_value_change3(0)

display(sidebyside)
display(slider_inner)
display(slider_outer)